In [1]:
from astropy.io import fits 
import matplotlib.pyplot as plt
import numpy as np 
from scipy.optimize import minimize, rosen, rosen_der
from scipy import interpolate
import scipy
import os 
from scipy.optimize import curve_fit
from scipy.integrate import simps
import math
from astropy.io import ascii
params = {"ytick.color" : "w",
          "xtick.color" : "w",
          "text.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "w"}
plt.rcParams.update(params)
from astropy.cosmology import WMAP9 as cosmo

In [7]:
U = ultra_fake[:,0]
V = ultra_fake[:,1]
J = ultra_fake[:,2]

In [9]:
all_outliers = [] 
for x in [[0,0.5,0.1], [0.5,1,0.1], [1,1.5,0.1], [1.5,2,1], [2,2.5,0.2]]:
    inds = np.where(np.logical_and(input_redshift > x[0], input_redshift < x[1]))
    #xray1 = Xhard[inds[0]]
    ids1 = ids[inds[0]]
    #q_or_sf1 = q_or_sf[inds[0]]
    #galaxies1 = ids[inds[0]]
    
    U_V = U-V
    V_J = V-J
    
    U_V = U_V[inds[0]]
    V_J = V_J[inds[0]]
    
    #removing nans where U V or J didn't exist 
    non_nan_inds = np.where(np.logical_and(np.isnan(U_V) == False , np.isnan(V_J) == False))[0]
    U_V = U_V[non_nan_inds]
    V_J = V_J[non_nan_inds]
    #xray1 = xray1[non_nan_inds]
    ids1 = ids1[non_nan_inds]
    #galaxies1 = galaxies1[non_nan_inds]

    quiescent = [] 
    
    first_section_V_J = np.where(np.logical_and(V_J > -6, V_J < 0.7))[0]
    first_section = np.where(U_V[first_section_V_J] > 1.3)[0]
    for fs in first_section: 
        quiescent.append(fs)

    second_section_V_J = np.where(np.logical_and(V_J > 0.7, V_J < 1.6))[0]
    for ss in second_section_V_J: 
        threshold = 0.7*(V_J[ss]-0.7)/0.9 + 1.3
        if U_V[ss] > threshold: 
            quiescent.append(ss)
    quiescent = np.array(quiescent) 
    
    V_J_outliers = np.where(np.logical_or(V_J > -1, V_J < 1))[0]
    
    U_V_outliers = np.where(np.logical_or(U_V > 2.5, U_V < -100))[0]
    
    for j in U_V_outliers: 
        if (j in V_J_outliers) == False: 
            np.append(V_J_outliers,j)
        else: 
            pass
        
    if (x[0] == 2) == True:
        all_outliers.append(ids1[U_V_outliers]) # used to be out of loop and be VJ also set bounds back
    else: 
        pass
    
    plt.clf()
    ax = plt.axes()
    cmap_reversed = plt.cm.get_cmap('ocean_r')
    xb = int((max(V_J) - min(V_J))*4)
    yb = int((max(U_V) - min(U_V))*4)
    
    plt.scatter(V_J, U_V, s=6)
    #print(min(xray1[np.isnan(xray1) == False]))
    #print(max(xray1[np.isnan(xray1) == False]))
    #plt.clim(0, .00001)
    #plt.colorbar()
    
    plt.xlabel('V-J')
    plt.ylabel('U-V')
    
    plt.plot([-6,0.7],[1.3,1.3],'k')
    plt.plot([0.7,1.6],[1.3,2],'k')
    plt.plot([1.6,1.6],[2,6],'k')
    plt.xlim(-1.5,2.5)
    plt.ylim(-0.5,3.5)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title('z=' + str(x[0]) + ',' + str(x[1]) + ' all plots each colorbar')
    plt.text(-1.25,-.25,str(int(len(quiescent)*100/len(U_V))) + '% Quiescent',color='black')
    plt.savefig('latest norm' + (str(x)) + '.png')
    plt.show()

IndexError: index 2601 is out of bounds for axis 0 with size 2597